In [ ]:
import pandas as pd
import numpy as np
from google.colab import files
from google.colab import drive
import ast
from pandas.io.json import json_normalize
import requests

In [ ]:
# Utilizo el método mount para acceder a los archivos situados en mi drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Creo los df a partir los csv

df_movies_csv = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/movies_dataset.csv')

<ipython-input-4-dc635a930292>:3: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_movies_csv = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/movies_dataset.csv')


In [ ]:
# Ejecuto este comando para poder visualizar todas las columnas cuando pido un .head() por ejemplo

pd.set_option('display.max_columns', None)

In [ ]:
# Voy a trabajar con el csv por lo tanto hago una copia sobre la que haré los cambios para resguardar el original

df_movies_csv_copy = df_movies_csv.copy()

In [ ]:
# Creo una función para crear diccionarios a partir de los strings en las columnas con información anidada

def convert_to_dict(x):
    try:
        return ast.literal_eval(x)
    except (ValueError, SyntaxError):
        return None

In [ ]:
# Aplico la función anterior en la columna 'belongs...'

df_movies_csv_copy['belongs_to_collection_dict'] = df_movies_csv_copy['belongs_to_collection'].apply(convert_to_dict)

In [ ]:
# Utilizo el método json_normalize para crear un nuevo df con los datos de la columna 'belongs...' desanidados

df_desanidado = pd.json_normalize(df_movies_csv_copy['belongs_to_collection_dict'])

In [ ]:
df_desanidado.rename(columns= {'id': 'id_collection', 'name': 'name_collection', 'poster_path': 'poster_path_collection', 'backdrop_path': 'backdrop_path_collection'}, inplace=True)

In [ ]:
# Concateno los dfs

df_movies_csv_copy = pd.concat([df_movies_csv_copy.drop('belongs_to_collection', axis=1), df_desanidado], axis=1)

In [ ]:
df_movies_csv_copy.drop('belongs_to_collection_dict', axis=1, inplace=True)

In [ ]:
# Comienzo a desanidar la columna 'production_companies'

df_movies_csv_copy['production_companies_dict'] = df_movies_csv_copy['production_companies'].apply(convert_to_dict)

In [ ]:
# Creo una función que servirá para obtener los valores que toma name en los diccionarios

def get_company_names(x):
    if isinstance(x, list) and len(x) > 0:
        return ', '.join([company['name'] for company in x])
    else:
        return None

In [ ]:
# Creo una columna con los valores de name aplicando en cada registro la función anterior

df_movies_csv_copy['name_company'] = df_movies_csv_copy['production_companies_dict'].apply(get_company_names)

In [ ]:
# Similar a la función anterior pero para obtener los id

def get_company_ids(x):
    if isinstance(x, list) and len(x) > 0:
        return ', '.join([str(company['id']) for company in x])
    else:
        return None

df_movies_csv_copy['id_company'] = df_movies_csv_copy['production_companies_dict'].apply(get_company_ids)

In [ ]:
df_movies_csv_copy = df_movies_csv_copy.drop(['production_companies', 'production_companies_dict'], axis=1)


In [ ]:
# Desanidación de la columna 'genres'

df_movies_csv_copy['genres_dict'] = df_movies_csv_copy['genres'].apply(ast.literal_eval)

def get_genre_names(x):
    return [genre['name'] for genre in x]

def get_genre_ids(x):
    return [genre['id'] for genre in x]

df_movies_csv_copy['genres_name'] = df_movies_csv_copy['genres_dict'].apply(get_genre_names)
df_movies_csv_copy['genres_id'] = df_movies_csv_copy['genres_dict'].apply(get_genre_ids)

df_movies_csv_copy = df_movies_csv_copy.drop(['genres', 'genres_dict'], axis=1)

In [ ]:
# Lo mismo para 'production_countries'

df_movies_csv_copy['production_countries_dict'] = df_movies_csv_copy['production_countries'].fillna('[]').apply(ast.literal_eval)

def get_iso_3166_1(x):
    if isinstance(x, list):
        return [country['iso_3166_1'] for country in x]
    else:
        return []

def get_country_names(x):
    if isinstance(x, list):
        return [country['name'] for country in x]
    else:
        return []

df_movies_csv_copy['production_countries_iso_3166_1'] = df_movies_csv_copy['production_countries_dict'].apply(get_iso_3166_1)
df_movies_csv_copy['production_countries_names'] = df_movies_csv_copy['production_countries_dict'].apply(get_country_names)

df_movies_csv_copy = df_movies_csv_copy.drop(['production_countries', 'production_countries_dict'], axis=1)

In [ ]:
# Lo mismo para 'spoken_languages'

df_movies_csv_copy['spoken_languages_dict'] = df_movies_csv_copy['spoken_languages'].fillna('[]').apply(ast.literal_eval)

def get_iso_639_1(x):
    if isinstance(x, list):
        return [language['iso_639_1'] for language in x]
    else:
        return []

def get_language_names(x):
    if isinstance(x, list):
        return [language['name'] for language in x]
    else:
        return []

df_movies_csv_copy['spoken_languages_iso_639_1'] = df_movies_csv_copy['spoken_languages_dict'].apply(get_iso_639_1)
df_movies_csv_copy['spoken_languages_name'] = df_movies_csv_copy['spoken_languages_dict'].apply(get_language_names)

df_movies_csv_copy = df_movies_csv_copy.drop(['spoken_languages', 'spoken_languages_dict'], axis=1)

In [ ]:
# Relleno con 0 los valores nulos de 'revenue'

df_movies_csv_copy['revenue'] = df_movies_csv_copy['revenue'].fillna(0)

In [ ]:
# Convierto los valores de 'release_date' a date

df_movies_csv_copy['release_date'] = pd.to_datetime(df_movies_csv_copy['release_date'], errors='coerce').dt.strftime('%Y-%m-%d').astype('datetime64')

In [ ]:
# Creo la columna 'release_year' a partir de los years de 'release_date'

df_movies_csv_copy['release_year'] = df_movies_csv_copy['release_date'].dt.year

In [ ]:
# Convierto a los NaN en 0 y paso toda la columna a int
df_movies_csv_copy['release_year'] = df_movies_csv_copy['release_year'].fillna(0).astype(int)

In [ ]:
# Paso a int los datos de 'budget'

df_movies_csv_copy['budget'] = pd.to_numeric(df_movies_csv_copy['budget'], errors='coerce')
df_movies_csv_copy['budget'] = df_movies_csv_copy['budget'].fillna(0).astype(int)

In [ ]:
# Creo la columna 'return' que es igual a 'revenue'/'budget'

df_movies_csv_copy['revenue'] = df_movies_csv_copy['revenue'].fillna(0).astype(int)
df_movies_csv_copy['return'] = df_movies_csv_copy.apply(lambda row: row['revenue'] / row['budget'] if row['budget'] != 0 else 0, axis=1)

In [ ]:
# Convierto la columna 'id' a int por si tengo que aplicar sort

df_movies_csv_copy['id'] = pd.to_numeric(df_movies_csv_copy['id'], errors='coerce')
df_movies_csv_copy['id'] = df_movies_csv_copy['id'].fillna(0).astype(int)

In [ ]:
df_movies_csv_copy.shape

(45466, 33)

In [ ]:
# Elimino los registros duplicados

df_movies_csv_copy = df_movies_csv_copy.drop_duplicates(subset=['id', 'title', 'popularity'])

In [ ]:
df_movies_csv_copy.shape


(45452, 33)

In [ ]:
# Capturo los title de los registros con valores faltantes en 'release_year'

peliculas_sin_year = df_movies_csv_copy[df_movies_csv_copy['release_year'] == 0]['title'].values

In [ ]:
# Trato de conseguir las fechas de lanzamiento de las peliculas que no tienen ese dato consultando a través de la api omdbapi

api_key = '914305fc'

for titulo in peliculas_sin_year:
    url = f'http://www.omdbapi.com/?apikey={api_key}&t={titulo}'
    response = requests.get(url)
    data = response.json()
    if 'Year' in data:
        df_movies_csv_copy.loc[df_movies_csv_copy['title'] == titulo, 'release_year'] = data['Year']
    else:
        df_movies_csv_copy.loc[df_movies_csv_copy['title'] == titulo, 'release_year'] = 0

In [ ]:
# Elimino las columnas que no se van a usar

df_movies_csv_copy.drop(columns=['video','imdb_id','adult','original_title','poster_path', 'homepage'], inplace=True)

In [ ]:
# Consulto cuántos registros aún quedan din dato en 'release_year'

df_movies_csv_copy[df_movies_csv_copy['release_year'] == 0]['title'].shape

(20,)

In [ ]:
# Elimino los 20 registros que quedaron sin dato en 'release_year'

df_movies_csv_copy = df_movies_csv_copy[df_movies_csv_copy['release_year'] != 0]

In [ ]:
# Elimino las filas donde omdbapi devolvió valores que no nos sirven para la columna 'release_year' (ejemplo: '2010 - 2020')

df_movies_csv_copy = df_movies_csv_copy[df_movies_csv_copy['release_year'].apply(lambda x: str(x).isdigit())]

In [ ]:
# Convierto los datos de la columna 'release_year' a enteros

df_movies_csv_copy['release_year'] = df_movies_csv_copy['release_year'].astype(int)

In [ ]:
df_movies_csv_copy.to_csv('/content/drive/MyDrive/Colab Notebooks/movies_dataset_limpio.csv', index=False)